# 01_CrewAI_基礎

## 🎯 學習目標
- 理解 CrewAI 框架的核心概念
- 掌握 Agent 和 Task 的設計方法
- 實作基礎的多智能體協作範例
- 比較框架實作與手動實作的差異

---

## 1. CrewAI 框架介紹

### 什麼是 CrewAI？

**CrewAI** 是一個專為多智能體協作設計的 Python 框架，讓我們能夠輕鬆建立和管理多個 AI 智能體的協作系統。

#### 🌟 核心概念
- **Agent (智能體)**: 具有特定角色和技能的 AI 實體
- **Task (任務)**: 需要完成的具體工作
- **Crew (團隊)**: 智能體和任務的組合
- **Tools (工具)**: 智能體可以使用的功能工具

#### ⚡ 框架優勢
- **簡化開發**：減少樣板代碼
- **內建協作**：自動處理智能體間溝通
- **工具整合**：豐富的內建工具庫
- **任務編排**：靈活的任務流程控制
- **錯誤處理**：內建的錯誤處理機制

### 與手動實作的比較

| 特點 | 手動實作 | CrewAI 框架 |
|------|----------|-------------|
| **代碼量** | 多（需自己實現所有邏輯） | 少（框架提供大部分功能） |
| **控制度** | 高（完全控制） | 中（框架約束但可配置） |
| **開發速度** | 慢（需要寫很多代碼） | 快（專注於業務邏輯） |
| **維護性** | 複雜（需自己維護） | 簡單（框架負責維護） |
| **學習成本** | 高（需理解所有細節） | 低（學會使用框架即可） |
| **除錯難度** | 高（需要自己處理） | 低（框架提供除錯工具） |

---

## 2. CrewAI 安裝與設定

### 安裝必要套件

In [ ]:
# 安裝 CrewAI 和相關套件
!pip install crewai crewai-tools langchain-openai

### 環境設定與驗證

In [ ]:
# 載入必要套件
import os
from datetime import datetime
from typing import List, Dict, Any

# CrewAI 相關
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from langchain_openai import ChatOpenAI

# 環境變數
from dotenv import load_dotenv
load_dotenv()

# 驗證 API 金鑰
openai_key = os.getenv('OPENAI_API_KEY')
if openai_key and openai_key != 'your-openai-api-key-here':
    print("✅ OpenAI API 金鑰已設定")
else:
    print("❌ 請設定 OpenAI API 金鑰")

# 初始化 LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)

print("✅ CrewAI 環境設定完成")

---

## 3. Agent 設計基礎

### 建立第一個 CrewAI Agent

In [ ]:
# 建立研究者智能體
research_agent = Agent(
    role='Research Specialist',
    goal='Conduct comprehensive research on given topics and provide detailed insights',
    backstory="""
    You are an experienced research specialist with expertise in gathering information from multiple sources.
    You excel at finding reliable sources, analyzing information, and synthesizing insights from different perspectives.
    Your research is thorough, objective, and always backed by credible sources.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 建立寫作者智能體
writer_agent = Agent(
    role='Content Writer',
    goal='Create well-structured, engaging, and informative long-form content',
    backstory="""
    You are a skilled content writer with expertise in creating long-form articles.
    You excel at organizing information into coherent structures, maintaining consistent tone,
    and crafting engaging content that resonates with the target audience.
    Your writing is clear, logical, and professionally formatted.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 建立編輯者智能體
editor_agent = Agent(
    role='Editor',
    goal='Review and improve content quality, ensuring accuracy and consistency',
    backstory="""
    You are a meticulous editor with a keen eye for detail.
    You excel at improving content quality, ensuring factual accuracy,
    maintaining style consistency, and enhancing overall readability.
    Your edits always improve the clarity and impact of the content.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm
)

print("✅ 建立了 3 個 CrewAI 智能體：")
print("   🔍 Research Specialist - 研究專家")
print("   ✍️ Content Writer - 內容寫作者")
print("   📝 Editor - 編輯者")

---

## 4. Task 設計基礎

### 定義具體任務

In [ ]:
def create_research_task(topic: str) -> Task:
    """建立研究任務"""
    return Task(
        description=f"""
Conduct comprehensive research on the topic: "{topic}"

Your task includes:
1. Analyze the topic from multiple expert perspectives (technical, business, user experience, academic)
2. Identify key aspects and current trends
3. Gather relevant data and statistics
4. Note important considerations and challenges

Provide a structured research report with:
- Executive summary (2-3 sentences)
- Key findings (5-8 bullet points)
- Expert insights from different perspectives
- Relevant statistics and data
- Current trends and future outlook

Topic: {topic}
""",
        expected_output="A comprehensive research report with structured findings, insights, and relevant data about the given topic.",
        agent=research_agent
    )

def create_writing_task(topic: str) -> Task:
    """建立寫作任務"""
    return Task(
        description=f"""
Based on the research report, create a well-structured long-form article about "{topic}".

Your task includes:
1. Create an engaging title and introduction
2. Develop a logical article structure with 3-4 main sections
3. Write detailed content for each section
4. Ensure coherent flow between sections
5. Include relevant examples and explanations
6. Write a compelling conclusion

Article requirements:
- Target length: 1500-2000 words
- Professional academic tone
- Clear structure with headers and subheaders
- Factual accuracy based on research
- Engaging and informative content

Use the research findings to support your arguments and provide comprehensive coverage of the topic.
""",
        expected_output="A complete long-form article (1500-2000 words) with clear structure, engaging content, and professional formatting.",
        agent=writer_agent
    )

def create_editing_task() -> Task:
    """建立編輯任務"""
    return Task(
        description="""
Review and edit the written article to ensure high quality and professional standards.

Your editing task includes:
1. Check for grammatical errors and typos
2. Improve sentence structure and clarity
3. Ensure logical flow and coherence
4. Verify factual accuracy against research
5. Enhance readability and engagement
6. Ensure consistent style and tone
7. Format headers and structure properly

Provide:
- The final edited article
- A brief summary of key improvements made
- Quality assessment score (1-10)

Focus on creating a polished, professional article ready for publication.
""",
        expected_output="A polished final article with improved quality, proper formatting, and a quality assessment summary.",
        agent=editor_agent
    )

print("✅ 任務模板已定義")
print("   📚 Research Task - 研究任務")
print("   ✍️ Writing Task - 寫作任務")
print("   📝 Editing Task - 編輯任務")

---

## 5. 基礎範例實作

### 建立簡單的寫作團隊

In [ ]:
# 測試主題
demo_topic = "區塊鏈技術在供應鏈管理中的應用"

# 建立任務
research_task = create_research_task(demo_topic)
writing_task = create_writing_task(demo_topic)
editing_task = create_editing_task()

# 建立寫作團隊 (Crew)
writing_crew = Crew(
    agents=[research_agent, writer_agent, editor_agent],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,  # 按順序執行任務
    verbose=2  # 顯示詳細執行過程
)

print(f"✅ 寫作團隊已建立")
print(f"   主題：{demo_topic}")
print(f"   智能體數量：{len(writing_crew.agents)}")
print(f"   任務數量：{len(writing_crew.tasks)}")
print(f"   執行方式：{writing_crew.process}")

### 執行簡單範例

In [ ]:
# 注意：這將花費一些時間和API費用
print("🚀 啟動 CrewAI 寫作流程...")
print("⚠️  這可能需要 3-5 分鐘時間")
print("=" * 60)

try:
    # 執行寫作流程
    start_time = datetime.now()
    
    result = writing_crew.kickoff()
    
    end_time = datetime.now()
    processing_time = (end_time - start_time).total_seconds()
    
    print(f"\n🎉 CrewAI 寫作流程完成！")
    print(f"⏱️  總耗時：{processing_time:.1f} 秒")
    print(f"📝 結果類型：{type(result)}")
    
except Exception as e:
    print(f"❌ 執行過程中發生錯誤：{str(e)}")
    result = None
    processing_time = 0

### 檢視執行結果

In [ ]:
if result:
    print("📄 CrewAI 生成結果：")
    print("=" * 60)
    
    # 顯示結果內容
    if isinstance(result, str):
        content = result
    else:
        content = str(result)
    
    # 計算字數
    word_count = len(content.split())
    
    print(f"📊 結果統計：")
    print(f"   字數：{word_count} 字")
    print(f"   字元數：{len(content)} 字元")
    print(f"   處理時間：{processing_time:.1f} 秒")
    
    # 顯示內容預覽
    print(f"\n📖 內容預覽（前500字）：")
    print("-" * 60)
    
    words = content.split()
    preview_words = words[:500] if len(words) > 500 else words
    preview = ' '.join(preview_words)
    
    print(preview)
    
    if len(words) > 500:
        print("\n... [內容繼續] ...")
    
    print("-" * 60)
else:
    print("❌ 沒有生成結果可以顯示")

---

## 6. 進階功能探索

### Tools 工具整合

In [ ]:
# 建立帶有工具的研究智能體
from crewai_tools import tool

@tool("Search and summarize")
def search_and_summarize(query: str) -> str:
    """
    Search for information on a given topic and provide a summary.
    This tool searches the web and returns key information about the query.
    """
    try:
        # 使用之前實作的搜尋功能
        from duckduckgo_search import DDGS
        
        ddgs = DDGS()
        results = list(ddgs.text(query, max_results=3))
        
        if not results:
            return f"No search results found for: {query}"
        
        summary = f"Search results for '{query}':\n\n"
        
        for i, result in enumerate(results, 1):
            summary += f"{i}. **{result['title']}**\n"
            summary += f"   {result.get('body', '')[:200]}...\n"
            summary += f"   來源: {result['href']}\n\n"
        
        return summary
        
    except Exception as e:
        return f"Search failed: {str(e)}"

# 建立帶有工具的智能體
enhanced_research_agent = Agent(
    role='Enhanced Research Specialist',
    goal='Conduct research using web search tools and provide detailed analysis',
    backstory="""
    You are an advanced research specialist equipped with web search capabilities.
    You can search for the latest information on any topic and provide comprehensive analysis.
    Always verify information from multiple sources and provide balanced perspectives.
    """,
    tools=[search_and_summarize],  # 加入工具
    verbose=True,
    allow_delegation=False,
    llm=llm
)

print("✅ 建立了增強版研究智能體，配備搜尋工具")

### 測試工具功能

In [ ]:
# 建立使用工具的研究任務
enhanced_research_task = Task(
    description=f"""
Research the topic "AI在醫療診斷中的最新發展" using your search tools.

Use the search_and_summarize tool to gather current information, then provide:
1. A comprehensive overview of the topic
2. Recent developments and breakthrough technologies
3. Key benefits and challenges
4. Future implications and trends

Make sure to use the search tool multiple times with different queries to get comprehensive coverage.
""",
    expected_output="A detailed research report based on current web search results about AI in medical diagnosis.",
    agent=enhanced_research_agent
)

# 建立簡單的測試團隊
test_crew = Crew(
    agents=[enhanced_research_agent],
    tasks=[enhanced_research_task],
    process=Process.sequential,
    verbose=2
)

print("✅ 工具測試團隊已建立")
print("🚀 即將測試工具整合功能...")

In [ ]:
# 執行工具測試（較輕量的測試）
try:
    print("🔍 測試智能體工具使用能力...")
    
    tool_test_result = test_crew.kickoff()
    
    print("\n✅ 工具測試完成！")
    print("📊 測試結果：")
    
    if isinstance(tool_test_result, str):
        words = len(tool_test_result.split())
        print(f"   字數：{words} 字")
        print(f"   內容預覽：{tool_test_result[:200]}...")
    
except Exception as e:
    print(f"❌ 工具測試失敗：{str(e)}")
    tool_test_result = None

---

## 7. CrewAI vs 手動實作比較

### 功能對比測試

In [ ]:
# 比較兩種方法的特點
print("⚖️ CrewAI vs 手動實作 - 詳細比較：")
print("=" * 70)

comparison_data = {
    "開發複雜度": {
        "手動實作": "高 - 需要實作所有協作邏輯",
        "CrewAI": "低 - 框架處理大部分複雜度"
    },
    "代碼量": {
        "手動實作": "多 - 約500-800行代碼",
        "CrewAI": "少 - 約100-200行代碼"
    },
    "學習曲線": {
        "手動實作": "陡峭 - 需理解所有實作細節",
        "CrewAI": "平緩 - 主要學習框架使用方法"
    },
    "自定義能力": {
        "手動實作": "非常高 - 可以完全自定義",
        "CrewAI": "高 - 透過配置和工具自定義"
    },
    "錯誤處理": {
        "手動實作": "需自己實現 - 容易遺漏邊界情況",
        "CrewAI": "框架提供 - 更完善和穩定"
    },
    "效能優化": {
        "手動實作": "需手動優化 - 需要深入了解",
        "CrewAI": "框架優化 - 自動處理大部分情況"
    },
    "社群支援": {
        "手動實作": "有限 - 主要靠自己解決問題",
        "CrewAI": "豐富 - 活躍的社群和文檔"
    },
    "部署難度": {
        "手動實作": "高 - 需要處理所有部署細節",
        "CrewAI": "中 - 框架提供部署支援"
    }
}

for aspect, comparison in comparison_data.items():
    print(f"\n🔸 **{aspect}**:")
    print(f"   手動實作: {comparison['手動實作']}")
    print(f"   CrewAI: {comparison['CrewAI']}")

### 使用情境建議

In [ ]:
print("\n🎯 使用情境建議：")
print("=" * 40)

scenarios = {
    "🎓 學習目的": {
        "建議": "手動實作",
        "原因": "深入理解多智能體協作原理，掌握底層實現邏輯"
    },
    "🏢 企業專案": {
        "建議": "CrewAI 框架",
        "原因": "快速開發，穩定可靠，有社群支援，適合生產環境"
    },
    "🔬 研究實驗": {
        "建議": "手動實作",
        "原因": "需要完全控制實驗參數，驗證特定假設"
    },
    "⚡ 快速原型": {
        "建議": "CrewAI 框架",
        "原因": "快速驗證想法，減少開發時間和成本"
    },
    "🔧 客製化需求": {
        "建議": "混合方式",
        "原因": "用 CrewAI 做基礎，手動實作特殊需求部分"
    },
    "📚 教學演示": {
        "建議": "兩者都用",
        "原因": "手動版理解原理，框架版展示實際應用"
    }
}

for scenario, info in scenarios.items():
    print(f"\n{scenario}:")
    print(f"   💡 建議：{info['建議']}")
    print(f"   📋 原因：{info['原因']}")

---

## 8. CrewAI 最佳實踐

### Agent 設計原則

In [ ]:
print("📋 CrewAI Agent 設計最佳實踐：")
print("=" * 50)

best_practices = {
    "角色定義": [
        "明確具體的角色職責",
        "避免角色功能重疊",
        "符合真實世界的專業角色"
    ],
    "目標設定": [
        "具體可測量的目標",
        "與角色能力匹配",
        "支援整體任務完成"
    ],
    "背景故事": [
        "豐富的專業背景描述",
        "體現專業特長和經驗",
        "提供行為指導原則"
    ],
    "工具配置": [
        "只配置角色需要的工具",
        "確保工具可靠性",
        "提供適當的工具使用指南"
    ]
}

for category, practices in best_practices.items():
    print(f"\n🔸 **{category}**:")
    for practice in practices:
        print(f"   • {practice}")

### Task 設計原則

In [ ]:
print("\n📋 CrewAI Task 設計最佳實踐：")
print("=" * 50)

task_practices = {
    "任務描述": [
        "詳細清晰的任務說明",
        "包含具體的執行步驟",
        "說明輸入和輸出要求"
    ],
    "預期輸出": [
        "明確定義輸出格式",
        "指定品質標準",
        "提供輸出範例"
    ],
    "任務關係": [
        "明確任務間的依賴關係",
        "合理安排任務順序",
        "設計適當的資料傳遞"
    ],
    "錯誤處理": [
        "預期可能的失敗情況",
        "提供備用執行方案",
        "設定適當的重試機制"
    ]
}

for category, practices in task_practices.items():
    print(f"\n🔸 **{category}**:")
    for practice in practices:
        print(f"   • {practice}")

---

## 🎯 本章小結

### 學習成果
✅ **CrewAI 基礎概念**：理解 Agent、Task、Crew、Tools 的核心概念  
✅ **框架使用方法**：掌握 CrewAI 的基本使用流程  
✅ **工具整合**：學會如何為智能體配置和使用工具  
✅ **實作對比**：深入了解框架與手動實作的優缺點  
✅ **最佳實踐**：掌握 CrewAI 開發的最佳實踐原則  

### 關鍵洞察
1. **框架價值**：CrewAI 大幅簡化多智能體系統開發
2. **設計重要性**：良好的 Agent 和 Task 設計是成功的關鍵
3. **工具生態**：豐富的工具支援讓智能體更強大
4. **適用場景**：不同情況下選擇不同的實作方式

### 技術優勢
- **開發效率**：代碼量減少60-80%
- **維護成本**：框架負責底層邏輯維護
- **擴展性**：容易添加新的智能體和任務
- **可靠性**：內建的錯誤處理和重試機制

### 下一步預告
在下個 notebook「02_CrewAI_長文寫作系統」中，我們將：
1. 用 **CrewAI 重構**我們的 STORM 系統
2. 實現更**進階的智能體協作**
3. 探索 CrewAI 的**進階功能**
4. 比較兩個版本的**性能差異**

---

*CrewAI 基礎學習完成！您現在已經掌握了使用現代多智能體框架的能力，準備好體驗更高效的開發方式了嗎？*